In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import *
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import metrics
from keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3
from itertools import combinations as comb
import ntpath
import glob
import math
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import OneHotEncoder 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/My Drive/"

/content/drive/My Drive


In [ ]:
data = './tumor/brain_tumor_dataset/'
No_brain_tumor = './tumor/brain_tumor_dataset/no/'
Yes_brain_tumor = './tumor/brain_tumor_dataset/yes/'
IMAGE_SIZE=200

In [ ]:
dirlist=[No_brain_tumor, Yes_brain_tumor]
classes=['No', 'Yes']
filepaths=[]
labels=[]
for i,j in zip(dirlist, classes):
    filelist=os.listdir(i)
    for f in filelist:
        filepath=os.path.join (i,f)
        filepaths.append(filepath)
        labels.append(j)
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))

filepaths:  253    labels:  253


In [ ]:
Files=pd.Series(filepaths, name='filepaths')
Label=pd.Series(labels, name='labels')
df=pd.concat([Files,Label], axis=1)
df=pd.DataFrame(np.array(df).reshape(253,2), columns = ['filepaths', 'labels'])
df.head()

,filepaths,labels
0,./tumor/brain_tumor_dataset/no/10 no.jpg,No
1,./tumor/brain_tumor_dataset/no/1 no.jpeg,No
2,./tumor/brain_tumor_dataset/no/11 no.jpg,No
3,./tumor/brain_tumor_dataset/no/39 no.jpg,No
4,./tumor/brain_tumor_dataset/no/33 no.jpg,No


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y = df['labels'].values
Y=le.fit_transform(Y)
Y.shape

(253,)

In [ ]:
X = []
for i, file in tqdm(enumerate(df['filepaths'].values)):
    img = image.load_img(df['filepaths'][i],target_size=(128, 128,3))
    imgp = image.img_to_array(img)
      #imgp=blurring(imgp)
    imgp = imgp/255
    X.append(imgp)
# Normalize the data
#X = X / 255.
X=np.array(X)
print('Train Shape: {}'.format(X.shape))

253it [00:01, 240.95it/s]


Train Shape: (253, 128, 128, 3)


In [ ]:
X=np.asarray(X).astype('float32')

In [ ]:
def Inceptionv3(class_no):
  print("-------------------------------------Inceptionv3--------------------------------------------")
  input_shape_densenet = (128, 128, 3)
  incept_model = tf.keras.applications.InceptionV3(include_top=False,weights="imagenet",input_shape=input_shape_densenet)
  incept_model.trainable = True
  
  x = tf.keras.layers.MaxPooling2D()(incept_model.output)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(512, activation='relu',kernel_regularizer='l2')(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  layer = tf.keras.layers.Dense(units=class_no,activation='sigmoid',kernel_regularizer='l2')(x)
  model = tf.keras.models.Model(incept_model.input, outputs=layer)
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),loss='binary_crossentropy',metrics=['acc'])
  #history = model.fit(train_imgs, train_labels, batch_size=32, epochs=num_epochs,verbose=0)
  #X_train_features,X_valid_features=incept_rf(incept_model,model,train_imgs,test_imgs,k)
  #result_imp,result_norm=imp_incept_rf(X_train_features,X_valid_features,train_labels)
  #print("------------------------------------------------------------------------------------------")
  return incept_model, model

In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=2, n_repeats=5, random_state=42)

In [ ]:
k=0
acc_imp=[]
acc_norm=[]
acc_incept=[]

imp_acc=[]
nor_acc=[]

for train_index, test_index in cv.split(X,Y):
    estimators_imp=[]
    estimators = []
    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y[train_index]
    Y_test = Y[test_index]
    
    imp_feature_size=1
    foz=4
    n_estimators=5
    num_class = 1
    imp_tree_predicts = []
    tree_predicts = []
    base_model,modely=Inceptionv3(num_class)
    callbacks =tf.keras.callbacks.EarlyStopping(patience=6, monitor='val_loss', verbose=1),
      

    history = modely.fit(X_train, Y_train,epochs=25,batch_size=16, validation_data=(X_test, Y_test),callbacks=[callbacks])
    
    pred= modely.predict(X_test)
    predicted_class=np.argmax(pred,axis=1)
    X_train_features,X_valid_features=incept_rf(base_model,modely,X_train,X_test,k)
    X_train_features = pd.DataFrame(X_train_features)
    X_valid_features = pd.DataFrame(X_valid_features)
    d=len(X_train_features.columns)
    imp_tree_predicts = []
    tree_predicts = []
    for i in range(0,n_estimators):
        imp_tr, imp_ts = generate_imp_space(X_train_features, Y_train, X_valid_features, imp_feature_size, foz)
        imp_d = imp_tr.shape[1]

      #meta learner params
        imp_sel_d = 2* round(math.log2(imp_d)) #feature
        sel_d = 2*round(math.log2(d))
      
        imp_tree = GradientBoostingClassifier(max_features=imp_sel_d, n_estimators=50,random_state=42)#,random_state=42
        #classifier = MultiOutputClassifier(imp_tree, n_jobs=-1)
        estimators_imp.append(('imp_rf'+str(i), imp_tree))
        imp_tree.fit(imp_tr, Y_train)
        imp_tree_predicts.append(imp_tree.predict(imp_ts))


        tree = GradientBoostingClassifier(max_features=sel_d, n_estimators=50,random_state=42)#, random_state=42
        estimators.append(('nor_rf'+str(i), tree))
        tree.fit(X_train_features, Y_train)
        tree_predicts.append(tree.predict(X_valid_features))
        
    voting_imp = VotingClassifier(estimators=estimators_imp)
    voting_imp.fit(imp_tr, Y_train)
    imp_pre=voting_imp.predict(imp_ts)
    voting_norm = VotingClassifier(estimators=estimators)
    voting_norm.fit(X_train_features, Y_train)
    imp_norm=voting_norm.predict(X_valid_features)
    #result_imp = MajorityVoting(np.array(imp_tree_predicts))
    #result_norm = MajorityVoting(np.array(tree_predicts))
    #act=np.argmax(Y_test,axis=1)
    imp_acc.append(metrics.accuracy_score(Y_test, imp_pre))
    nor_acc.append(metrics.accuracy_score(Y_test, imp_norm))
    #acc_imp.append(metrics.accuracy_score(Y_test, result_imp))
    #acc_norm.append(metrics.accuracy_score(Y_test, result_norm))
    acc_incept.append(metrics.accuracy_score(Y_test, predicted_class))
    k+=1

-------------------------------------Inceptionv3--------------------------------------------
Epoch 1/25
8/8 [==============================] - 13s 404ms/step - loss: 8.9690 - acc: 0.5794 - val_loss: 9.8991 - val_acc: 0.6063
Epoch 2/25
8/8 [==============================] - 1s 84ms/step - loss: 8.7045 - acc: 0.6746 - val_loss: 10.4915 - val_acc: 0.6063
Epoch 3/25
8/8 [==============================] - 1s 82ms/step - loss: 8.4714 - acc: 0.7619 - val_loss: 10.8431 - val_acc: 0.6063
Epoch 4/25
8/8 [==============================] - 1s 84ms/step - loss: 8.2444 - acc: 0.8175 - val_loss: 10.9939 - val_acc: 0.6063
Epoch 5/25
8/8 [==============================] - 1s 82ms/step - loss: 8.1272 - acc: 0.8968 - val_loss: 11.0751 - val_acc: 0.6063
Epoch 6/25
8/8 [==============================] - 1s 82ms/step - loss: 7.9561 - acc: 0.8968 - val_loss: 11.1173 - val_acc: 0.6063
Epoch 7/25
8/8 [==============================] - 1s 83ms/step - loss: 7.8806 - acc: 0.9286 - val_loss: 10.7227 - val_acc: 0.6

In [ ]:
print("RF")
print(np.mean(imp_acc))
print(np.mean(nor_acc))
print(np.mean(acc_incept))

RF
0.8339582552180979
0.8363329583802026
0.3873328333958255


In [ ]:
print("XG")
print(np.mean(imp_acc))
print(np.mean(nor_acc))
print(np.mean(acc_incept))

XG
0.7960567429071367
0.7960567429071367
0.3873328333958255


In [ ]:
print("GB")
print(np.mean(imp_acc))
print(np.mean(nor_acc))
print(np.mean(acc_incept))

GB
0.8347769028871392
0.8339582552180979
0.3873328333958255


In [ ]:
def generate_imp_space(X_train, Y_train, X_test, imp_feature_size, foz):
  imp_train_data = X_train.values
  imp_test_data = X_test.values
  Y_train=pd.DataFrame(Y_train)
  d = len(X_train.columns)
  # print("____1_____")
  for i in range(0,imp_feature_size*foz):
    Xindis = np.random.permutation(d)
    for j in range(0,d-(foz-1),foz):  #d/foz kadar doner
      sX = np.random.permutation(num_class)
      s1 = sX[0]
      # print("____2_____")
      s1data = X_train[X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s2data = X_train[~X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s1data = s1data.iloc[:,Xindis[j:j+(foz)]]
      s2data = s2data.iloc[:,Xindis[j:j+(foz)]] # s1 vs all other classes, #foz feature
      # print("____3_____")
      s1label = np.ones((s1data.values.shape[0],1),dtype=int)
      s2label = -1*np.ones((s2data.values.shape[0],1),dtype=int)
      Wdata = np.concatenate((s1data,s2data))
      # print("____4_____")
      
      Wdata = x2fx(Wdata)
      Wlabel = np.concatenate((s1label,s2label))
      W = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Wdata.T, Wdata)),Wdata.T),Wlabel)
      
      WW = x2fx(X_train.iloc[:,Xindis[j:j+(foz)]].values)
      imp_train_data = np.concatenate((imp_train_data, np.matmul(WW,W)),axis=1)
      
      TT = x2fx(X_test.iloc[:,Xindis[j:j+(foz)]].values)
      imp_test_data = np.concatenate((imp_test_data, np.matmul(TT,W)),axis=1)
    
  return imp_train_data,imp_test_data
  

In [ ]:
def incept_rf(base_model,modely,train_imgs,test_imgs,i):
  X_train_features = []
  X_valid_features = []
  if(i==0):

    feature_network = Model(base_model.input, modely.get_layer('flatten').output)
  else:
    feature_network = Model(base_model.input, modely.get_layer('flatten_'+str(i)).output)
  X_train_features = feature_network.predict(train_imgs)  # Assuming you have your images in x
  X_valid_features = feature_network.predict(test_imgs)
  return X_train_features,X_valid_features


In [ ]:
def x2fx(x, model='quadratic'):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == 'linear':
        return linear
    if model == 'purequadratic':
        return np.c_[linear, x**2]
    interaction = np.array([x[:,i]*x[:,j] for i, j in comb(range(x.shape[1]), 2)]).T
    if model == 'interaction':
        return np.c_[linear, interaction]
    if model == 'quadratic':
        return np.c_[linear, interaction, x**2]


In [ ]:
def MajorityVoting(votes):
  results = []
  for i in range(0,votes.shape[1]):
    values, counts = np.unique(votes[:,i], return_counts=True)
    print(counts)
    results.append(values[np.argmax(counts)])
  return np.array(results)